In [ ]:
from dsc80_utils import *
import lec16_util as util

# Lecture 16 – Hyperparameters, Cross-Validation, and Decision Trees

## DSC 80, Winter 2024

### Announcements 📣

- Project 4 is released! Read all about it at [**dsc80.com/proj04**](https://dsc80.com/proj04).
    - The checkpoint is due **this Thursday**.
    - The full project is due on **Thursday, March 21st**. You **cannot** use slip days on the final deadline.
- Lab 9 is due on **Monday, March 11th**.
- The Final Exam is on **Tuesday, March 19th from 3-6PM** (room TBD).
    - Practice by working through old exams at [practice.dsc80.com](https://practice.dsc80.com). Even more exams are there now, including this quarter's midterm.
    - You can bring two double-sided notes sheets (you can bring your midterm notes sheet, if you want).
    - More details to come.

### Agenda 📆

- Hyperparameters.
- Cross-validation.
- Decision trees.

<div class="alert alert-warning">
    <h3>Question 🤔 (Answer at <a href="https://docs.google.com/forms/d/e/1FAIpQLScWbVZv9hBv-wX-ItKHUVRnkPMMtfJZVfErKE9GS7_8dFcRBQ/viewform">q.dsc80.com)</h3>
</div>
    
Remember, you can always ask questions at [**q.dsc80.com**](https://docs.google.com/forms/d/e/1FAIpQLScWbVZv9hBv-wX-ItKHUVRnkPMMtfJZVfErKE9GS7_8dFcRBQ/viewform)!

## Hyperparameters

In [ ]:
np.random.seed(23) # For reproducibility.

def sample_from_pop(n=100):
    x = np.linspace(-2, 3, n)
    y = x ** 3 + (np.random.normal(0, 3, size=n))
    return pd.DataFrame({'x': x, 'y': y})

sample_1 = sample_from_pop()
sample_2 = sample_from_pop()

In [ ]:
px.scatter(sample_1, x='x', y='y', title='Sample 1')

### Example: Polynomial regression

We recently looked at an example of **polynomial regression**.

In [ ]:
fig = util.train_and_plot(train_sample=sample_1, test_sample=sample_2, degs=[1, 3, 25], data_name='Sample 2')
fig.update_layout(title='Trained on Sample 1, Performance on Sample 2')

When building these models:
- We **got to choose** the degree of the polynomials (i.e. we chose 1, 3, and 25).
- We didn't get to choose the exact formulas for the three polynomials – their formulas were **learned from data**.

### Parameters vs. hyperparameters

- A **parameter** defines the relationship between variables in a model. 
    - **We learn parameters from data**.
    - For instance, suppose we fit a degree 3 polynomial to data, and end up with
    
    $$H(x) = 1 - 2x + 13x^2 - 4x^3$$
    
    - 1, -2, 13, and -4 are parameters.

- A **hyperparameter** is a parameter that we get to choose **before our model is fit to the data**.
    - Think of hyperparameters as knobs 🎛 – **we get to pick and tune them!**
    - **Polynomial degree** was a hyperparameter in the previous example, and we tried three different values: 1, 3, and 25.

- **Question**: How do we choose the "right" hyperparameter(s)?

### Training error vs. test error

- We know that a model's performance on a **test set** is a good estimate of its ability to generalize to unseen data.

- We want to find the hyperparameter that leads to the best **test set performance**.

- Idea:
    1. Come up with a **list** of hyperparameters to try.
    2. For each hyperparameter, train the model on the training set and compute its performance on the test set.
    3. Pick the hyperparameter with the best performance on the test set.

### Training error vs. test error

- Let's try this strategy on Sample 1 from our earlier example. 

- We'll try to fit a polynomial model on the dataset; we'll choose the polynomial's degree from the list [1, 2, ..., 25].

In [ ]:
px.scatter(sample_1, x='x', y='y', title='Sample 1')

First, we perform a train-test split.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = sample_1[['x']]
y = sample_1['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=100)

### 💡 Pro-Tip: Using `make_pipeline`

Instead of using the `Pipeline` class directly, you can use `make_pipeline` for a more compact syntax:

(Also, there's an analogous `sklearn.compose.make_column_transformer` for the `ColumnTransformer` class.)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
# Pipeline with degree-2 polynomial features
pl = Pipeline([('poly', PolynomialFeatures(2)), ('lin-reg', LinearRegression())])

# Same pipeline, notice that make_pipeline generates names for pipeline steps
pl = make_pipeline(PolynomialFeatures(2), LinearRegression())
pl

### Polynomial degree vs. train/test error

Now, we'll create models with degree-1 through degree-25 polynomial features and compute their train and test errors.

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
train_errs = []
test_errs = []

for d in range(1, 26):
    pl = make_pipeline(PolynomialFeatures(d), LinearRegression())
    pl.fit(X_train, y_train)
    train_errs.append(mean_squared_error(y_train, pl.predict(X_train), squared=False))
    test_errs.append(mean_squared_error(y_test, pl.predict(X_test), squared=False))

errs = pd.DataFrame({'Train Error': train_errs, 'Test Error': test_errs})

Let's look at the plots of training error vs. degree and test error vs. degree.

In [ ]:
fig = px.line(errs)
fig.update_layout(showlegend=True, xaxis_title='Polynomial Degree', yaxis_title='RMSE')

- Training error appears to decrease as polynomial degree increases.

- Test error appears to decrease until a "valley", and then increases again.

- Here, we'd choose a degree of 3, since that degree has the **lowest test error**.

### Training error vs. test error

The pattern we saw in the previous example is true more generally.

<center><img src='imgs/tt-errors.png' width=50%></center>

We pick the hyperparameter(s) at the "valley" of test error.

Note that training error **tends** to underestimate test error, but it doesn't have to – i.e., it is possible for test error to be lower than training error (say, if the test set is "easier" to predict than the training set).

### Conducting train-test splits

- Recall, <span style='color: blue'><b>training data</b></span> is used to fit our model, and <span style='color: orange'><b>test data</b></span> is used to evaluate our model.

<center><img src='imgs/train-test-first.png' width=40%></center>

- **Question**: _How_ should we split?
    - `sklearn`'s `train_test_split` splits **randomly**, which usually works well.
    - However, if there is some element of **time** in the training data (say, when predicting the future price of a stock), a better split is "past" and "future".

- **Question**: How _large_ should the split be, e.g. 90%-10% vs. 75%-25%?
    - There's a tradeoff – a larger training set should lead to a "better" model, while a larger test set should lead to a better estimate of our model's ability to generalize.
    - There's no "right" choice, but we usually choose between 10% to 25% for the test set.

### But wait...

- With our current strategy, we are choosing the hyperparameter that creates the model that **performs best on the test set**.

- As such, we are **overfitting to the test set** – the best hyperparameter for the test set might not be the best hyperparameter for a totally unseen dataset!

- It seems like we need **another** split.

## Cross-validation

### Idea: A single validation set

<center><img src='imgs/train-test-val.png' width=40%></center>

1. Split the data into three sets: <span style='color: blue'><b>training</b></span>, <span style='color: green'><b>validation</b></span>, and <span style='color: orange'><b>test</b></span>.

2. For each hyperparameter choice, <span style='color: blue'><b>train</b></span> the model only on the <span style='color: blue'><b>training set</b></span>, and <span style='color: green'><b>evaluate</b></span> the model's performance on the <span style='color: green'><b>validation set</b></span>.

3. Find the hyperparameter with the best <span style='color: green'><b>validation</b></span> performance.

4. Retrain the final model on the <span style='color: blue'><b>training</b></span> and <span style='color: green'><b>validation</b></span> sets, and report its performance on the <span style='color: orange'><b>test set</b></span>.

**Issue**: This strategy is too dependent on the <span style='color: green'><b>validation</b></span> set, which may be small and/or not a representative sample of the data. **We're not going to do this.** ❌

### A better idea: $k$-fold cross-validation

Instead of relying on a single <span style='color: green'><b>validation</b></span> set, we can create $k$ <span style='color: green'><b>validation</b></span> sets, where $k$ is some positive integer (5 in the example below).

<center><img src='imgs/k-fold.png' width=40%></center>

Since each data point is used for <span style='color: blue'><b>training</b></span> $k-1$ times and <span style='color: green'><b>validation</b></span> once, the (averaged) <span style='color: green'><b>validation</b></span> performance should be a good metric of a model's ability to generalize to unseen data.

$k$-fold cross-validation (or simply "cross-validation") is **the** technique we will use for finding hyperparameters, or more generally, for choosing between different possible models. **It's what you should use in Project 4!** ✅

### Creating folds in `sklearn`

`sklearn` has a `KFold` class that splits data into training and validation folds.

In [ ]:
from sklearn.model_selection import KFold

Let's use a simple dataset for illustration.

In [ ]:
data = np.arange(10, 70, 10)
data

Let's instantiate a `KFold` object with $k=3$.

In [ ]:
kfold = KFold(3, shuffle=True, random_state=1)
kfold

Finally, let's use `kfold` to `split` `data`:

In [ ]:
for train, val in kfold.split(data):
    print(f'train: {data[train]}, validation: {data[val]}')

Note that each value in `data` is used for validation exactly once and for training exactly twice. Also note that because we set `shuffle=True` the folds are not simply `[10, 20]`, `[30, 40]`, and `[50, 60]`.

### $k$-fold cross-validation

First, **shuffle** the entire training set randomly and **split** it into $k$ disjoint folds, or "slices". Then:

- For each hyperparameter:
    - For each slice:
        - Let the slice be the "validation set", $V$.
        - Let the rest of the data be the "training set", $T$.
        - Train a model using the selected hyperparameter on the training set $T$.
        - Evaluate the model on the validation set $V$.
    - Compute the **average** validation score (e.g. RMSE) for the particular hyperparameter.

- Choose the hyperparameter with the best average validation score.

### $k$-fold cross-validation in `sklearn`

While you could manually use `KFold` to perform cross-validation, the `cross_val_score` function in `sklearn` implements $k$-fold cross-validation for us! 

```py
cross_val_score(estimator, X_train, y_train, cv)
```

Specifically, it takes in:
- A `Pipeline` or estimator **that has not already been `fit`**.
- Training data.
- A value of $k$ (through the `cv` argument).
- (Optionally) A `scoring` metric.

and performs $k$-fold cross-validation, returning the values of the scoring metric on each fold.

In [ ]:
from sklearn.model_selection import cross_val_score

### $k$-fold cross-validation in `sklearn`

- Let's perform $k$-fold cross validation in order to help us pick a degree for polynomial regression from the list [1, 2, ..., 25].

- We'll use $k=5$ since it's a common choice (and the default in `sklearn`).

- For the sake of this example, we'll suppose `sample_1` is our "training + validation data", i.e. that our test data is in some other dataset.
    - If this were not true, we'd first need to split `sample_1` into separate training and test sets.

In [ ]:
errs_df = pd.DataFrame()

for d in range(1, 26):
    pl = make_pipeline(PolynomialFeatures(d), LinearRegression())
    
    # The `scoring` argument is used to specify that we want to compute the RMSE; 
    # the default is R^2. It's called "neg" RMSE because, 
    # by default, sklearn likes to "maximize" scores, and maximizing -RMSE is the same
    # as minimizing RMSE.
    errs = cross_val_score(pl, sample_1[['x']], sample_1['y'], 
                           cv=5, scoring='neg_root_mean_squared_error')
    errs_df[f'Deg {d}'] = -errs # Negate to turn positive (sklearn computed negative RMSE).
    
errs_df.index = [f'Fold {i}' for i in range(1, 6)]
errs_df.index.name = 'Validation Fold'

Next class, we'll look at how to implement this procedure without needing to `for`-loop over values of `d`.

### $k$-fold cross-validation in `sklearn`

Note that for each choice of degree (our hyperparameter), we have **five** RMSEs, one for each "fold" of the data. This means that in total, $5 \cdot 25 = 125$ models were trained/fit to data!

In [ ]:
errs_df

We should choose the degree with the lowest **average** validation RMSE.

In [ ]:
errs_df.mean().idxmin()

Note that if we didn't perform $k$-fold cross-validation, but instead just used a single validation set, we may have ended up with a different result:

In [ ]:
errs_df.idxmin(axis=1)

***Note***: You may notice that the RMSEs in Folds 1 and 5 are much higher than in other folds. Can you think of reasons why, and how we might fix this?

In [ ]:
px.scatter(sample_1, x='x', y='y', title='Sample 1')

### Another example: Tips

We can also use $k$-fold cross-validation to determine which subset of features to use in a linear model that predicts tips by making one pipeline for each subset of features we want to evaluate.

In [ ]:
# make_column_transformer is a shortcut for the ColumnTransformer class
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder

As we should always do, we'll perform a train-test split on `tips` and will only use the training data for cross-validation.

In [ ]:
tips = sns.load_dataset('tips')
tips.head()

In [ ]:
X = tips.drop('tip', axis=1)
y = tips['tip']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
# A dictionary that maps names to Pipeline objects.
pipes = {
    'total_bill only': make_pipeline(
        make_column_transformer(
            (FunctionTransformer(lambda x: x), ['total_bill']),
        ),
        LinearRegression(),
    ),
    'total_bill + size': make_pipeline(
        make_column_transformer(
            (FunctionTransformer(lambda x: x), ['total_bill', 'size']),
        ),
        LinearRegression(),
    ),
    'total_bill + size + OHE smoker': make_pipeline(
        make_column_transformer(
            (FunctionTransformer(lambda x: x), ['total_bill', 'size']),
            (OneHotEncoder(drop='first'), ['smoker']),
        ),
        LinearRegression(),
    ),
    'total_bill + size + OHE all': make_pipeline(
        make_column_transformer(
            (FunctionTransformer(lambda x: x), ['total_bill', 'size']),
            (OneHotEncoder(drop='first'), ['smoker', 'sex', 'time', 'day']),
        ),
        LinearRegression(),
    ),
}

In [ ]:
pipe_df = pd.DataFrame()

for pipe in pipes:
    errs = cross_val_score(pipes[pipe], X_train, y_train,
                           cv=5, scoring='neg_root_mean_squared_error')
    pipe_df[pipe] = -errs
    
pipe_df.index = [f'Fold {i}' for i in range(1, 6)]
pipe_df.index.name = 'Validation Fold'

In [ ]:
pipe_df

In [ ]:
pipe_df.mean()

In [ ]:
pipe_df.mean().idxmin()

Even though the third model has the lowest average validation RMSE, its average validation RMSE is very close to that of the other, simpler models, and as a result we'd likely use the simplest model in practice.

<div class="alert alert-success" markdown="1">

<h3>Exercise</h3>
    
- Suppose you have a training dataset with 1000 rows.
- You want to decide between 20 hyperparameters for a particular model.
- To do so, you perform 10-fold cross-validation.
- **How many times is the first row in the training dataset (`X.iloc[0]`) used for training a model?**

</div>

### Summary: Generalization

1. Split the data into two sets: <span style='color: blue'><b>training</b></span> and <span style='color: orange'><b>test</b></span>.

2. Use only the <span style='color: blue'><b>training</b></span> data when designing, training, and tuning the model.
    - Use <span style='color: green'><b>$k$-fold cross-validation</b></span> to choose hyperparameters and estimate the model's ability to generalize.
    - Do not ❌ look at the <span style='color: orange'><b>test</b></span> data in this step!

3. Commit to your final model and train it using the entire <span style='color: blue'><b>training</b></span> set.

4. Test the data using the <span style='color: orange'><b>test</b></span> data. If the performance (e.g. RMSE) is not acceptable, return to step 2.

5. Finally, train on **all available data** and ship the model to production! 🛳

🚨 This is the process you should **always** use! 🚨 

<div class="alert alert-warning">
    <h3>Question 🤔 (Answer at <a href="https://docs.google.com/forms/d/e/1FAIpQLScWbVZv9hBv-wX-ItKHUVRnkPMMtfJZVfErKE9GS7_8dFcRBQ/viewform">q.dsc80.com)</h3>
</div>
    
Remember, you can always ask questions at [**q.dsc80.com**](https://docs.google.com/forms/d/e/1FAIpQLScWbVZv9hBv-wX-ItKHUVRnkPMMtfJZVfErKE9GS7_8dFcRBQ/viewform)!

## Decision trees 🌲

<center><img src='imgs/ml-taxonomy.svg' width=50%></center>

Decision trees can be used for both regression and classification. We'll start by using them for **classification**.

### Example: Should I get groceries?

Decision trees make classifications by answering a series of yes/no questions.

<center><img src='imgs/dtree-basic.svg' width=400></center>

Internal **nodes** of trees involve questions; leaf nodes make predictions $H(x)$.

<center><img src='imgs/dtree-basic-plot.svg' width=400></center>

### Example: Predicting diabetes

In [ ]:
diabetes = pd.read_csv(Path('data') / 'diabetes.csv')
display_df(diabetes, cols=9)

In [ ]:
# 0 means no diabetes, 1 means yes diabetes.
diabetes['Outcome'].value_counts()

- `'Glucose'` is measured in mg/dL (milligrams per deciliter).

- `'BMI'` is calculated as $\text{BMI} = \frac{\text{weight (kg)}}{\left[ \text{height (m)} \right]^2}$.

- Let's use `'Glucose'` and `'BMI'` to predict whether or not a patient has diabetes (`'Outcome'`).

### Exploring the dataset

First, a train-test split:

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = (
    train_test_split(diabetes[['Glucose', 'BMI']], diabetes['Outcome'], random_state=1)
)

<span style='color: orange'><b>Class 0 (orange) is "no diabetes"</b></span> and <span style='color: blue'><b>class 1 (blue) is "diabetes"</b></span>.

In [ ]:
fig = (
    X_train.assign(Outcome=y_train.astype(str))
            .plot(kind='scatter', x='Glucose', y='BMI', color='Outcome', 
                  color_discrete_map={'0': 'orange', '1': 'blue'},
                  title='Relationship between Glucose, BMI, and Diabetes')
)
fig

### Building a decision tree

Let's build a decision tree and interpret the results.

The relevant class is `DecisionTreeClassifier`, from `sklearn.tree`.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

Note that we `fit` it the same way we `fit` earlier estimators.

_You may wonder what `max_depth` and `criterion` do – more on this soon!_

In [ ]:
dt = DecisionTreeClassifier(max_depth=2, criterion='entropy')

In [ ]:
dt.fit(X_train, y_train)

### Visualizing decision trees

Our fit decision tree is like a "flowchart", made up of a series of questions.

As before, <span style='color: orange'><b>orange is "no diabetes"</b></span> and <span style='color: blue'><b>blue  is "diabetes"</b></span>.

In [ ]:
from sklearn.tree import plot_tree

In [ ]:
plt.figure(figsize=(15, 5))
plot_tree(dt, feature_names=X_train.columns, class_names=['no db', 'yes db'], 
          filled=True, fontsize=15, impurity=False);

- To **classify a new data point**, we start at the top and answer the first question (i.e. "Glucose <= 129.5").

- If the answer is "**Yes**", we move to the **left** branch, otherwise we move to the right branch.

- We repeat this process until we end up at a leaf node, at which point we predict the most common class in that node.
    - Note that each node has a `value` attribute, which describes the number of **training** individuals of each class that fell in that node.

In [ ]:
# Note that the left node at depth 2 has a `value` of [304, 78].
y_train[X_train.query('Glucose <= 129.5').index].value_counts()

### Evaluating classifiers

The most common evaluation metric in classification is **accuracy**:

$$\text{accuracy} = \frac{\text{# data points classified correctly}}{\text{# data points}}$$

In [ ]:
(dt.predict(X_train) == y_train).mean()

The `score` method of a classifier computes accuracy by default (just like the `score` method of a regressor computes $R^2$ by default). We want our classifiers to have **high accuracy**.

In [ ]:
# Training accuracy – same number as above
dt.score(X_train, y_train)

In [ ]:
# Testing accuracy
dt.score(X_test, y_test)

### Reflection

- Decision trees are easily interpretable: it is clear _how_ they make their predictions.

- They work with categorical data without needing to use one hot encoding.

- They also can be used in multi-class classification problems, e.g. when there are more than 2 possible outcomes.

- The _decision boundary_ of a decision tree can be arbitrarily complicated.

- **How are decision trees trained?**

### How are decision trees trained?

Pseudocode:

```python
def make_tree(X, y):
    if all points in y have the same label C:
        return Leaf(C)
    f = best splitting feature # e.g. Glucose or BMI
    v = best splitting value   # e.g. 129.5
    
    X_left, y_left   = X, y where (X[f] <= v)
    X_right, y_right = X, y where (X[f] > v)
    
    left  = make_tree(X_left, y_left)
    right = make_tree(X_right, y_right)
    
    return Node(f, v, left, right)
    
make_tree(X_train, y_train)
```

### How do we measure the quality of a split?

Our pseudocode for training a decision tree relies on finding the best way to "split" a node – that is, **the best question to ask** to help us narrow down which class to predict.

**Intuition**: Suppose the distribution within a node looks like this (colors represent classes):

<center>🟠🟠🟠🟠🟠🟠🔵🔵🔵🔵🔵🔵🔵</center>

Split A:
- "Yes": 🟠🟠🟠🔵🔵🔵
- "No": 🟠🟠🟠🔵🔵🔵🔵

Split B:
- "Yes": 🔵🔵🔵🔵🔵🔵
- "No": 🔵🟠🟠🟠🟠🟠🟠

**Which split is "better"?**

Split B, because there is "less uncertainty" in the resulting nodes in Split B than there is in Split A. Let's try and quantify this!

### Entropy

- For each class $C$ within a node, define $p_C$ as the proportion of points with that class.
    - For example, the two classes may be "yes diabetes" and "no diabetes".

- The **surprise** of drawing a point from the node at random and having it be class $C$ is:

$$
- \log_2 p_C
$$

- The **entropy** of a node is the average surprise over all classes:

\begin{align}
\text{entropy} &= - \sum_C p_C \log_2 p_C
\end{align}

- The entropy of 🟠🟠🟠🟠🟠🟠🟠🟠 is $ -1 \log_2(1) = 0 $.

- The entropy of 🟠🟠🟠🟠🔵🔵🔵🔵 is $ -0.5 \log_2(0.5) - 0.5 \log_2(0.5) = 1$.

- The entropy of 🟠🔵🟢🟡🟣 is $ - \log_2 \frac{1}{5} = \log_2(5) $
    - In general, if a node has $n$ points, all with different labels, the entropy of the node is $ \log_2(n) $.

### Example entropy calculation

Suppose we have:

<center>🟠🟠🟠🟠🟠🟠🟠🟠🟠🟠🟠🟠🔵🔵🔵🔵🔵🔵</center>

Split A:
- "Yes": 🟠🟠🟠🟠🟠🟠🔵
- "No": 🟠🟠🟠🟠🟠🟠🔵🔵🔵🔵🔵

Split B:
- "Yes": 🟠🟠🟠🟠🟠🟠🔵🔵🔵
- "No": 🟠🟠🟠🟠🟠🟠🔵🔵🔵

We choose the split that has the lowest **weighted entropy**, that is:

$$\text{entropy of split} = \frac{\# \text{Yes}}{\# \text{Yes} + \# \text{No}} \cdot \text{entropy(Yes)} + \frac{\# \text{No}}{\# \text{Yes} + \# \text{No}} \cdot \text{entropy(No)}$$

In [ ]:
def entropy(node):
    props = pd.Series(list(node)).value_counts(normalize=True)
    return -sum(props * np.log2(props))

In [ ]:
def weighted_entropy(yes_node, no_node):
    yes_entropy = entropy(yes_node)
    no_entropy = entropy(no_node)
    yes_weight = len(yes_node) / (len(yes_node) + len(no_node))
    return yes_weight * yes_entropy + (1 - yes_weight) * no_entropy

In [ ]:
# Split A:
weighted_entropy("🟠🟠🟠🟠🟠🟠🔵", "🟠🟠🟠🟠🟠🟠🔵🔵🔵🔵🔵")

In [ ]:
# Split B:
weighted_entropy("🟠🟠🟠🟠🟠🟠🔵🔵🔵", "🟠🟠🟠🟠🟠🟠🔵🔵🔵")

Split A has the lower weighted entropy, so we'll use it.

### Understanding entropy

In [ ]:
plt.figure(figsize=(15, 5))
plot_tree(dt, feature_names=X_train.columns, class_names=['no db', 'yes db'], 
          filled=True, fontsize=15, impurity=True);

We can recreate the entropy calculations above! Note that the default `DecisionTreeClassifier` uncertaintly metric _isn't_ entropy; it used entropy because we set `criterion='entropy'` when defining `dt`. (The default metric, [Gini impurity](https://en.wikipedia.org/wiki/Decision_tree_learning#:~:text=Gini%20impurity%20measures%20how%20often,into%20a%20single%20target%20category.), is perfectly fine too!)

In [ ]:
# The first node at depth 2 has an entropy of 0.73,
# both told to us above and verified here!
entropy([0] * 304 + [1] * 78)

<div class="alert alert-warning">
    <h3>Question 🤔 (Answer at <a href="https://docs.google.com/forms/d/e/1FAIpQLScWbVZv9hBv-wX-ItKHUVRnkPMMtfJZVfErKE9GS7_8dFcRBQ/viewform">q.dsc80.com)</h3>
</div>
    
Remember, you can always ask questions at [**q.dsc80.com**](https://docs.google.com/forms/d/e/1FAIpQLScWbVZv9hBv-wX-ItKHUVRnkPMMtfJZVfErKE9GS7_8dFcRBQ/viewform)!

### Tree depth

Decision trees are trained by **recursively** picking the best split until:

- all "leaf nodes" only contain training examples from a single class (good), or
- it is impossible to split leaf nodes any further (not good).

By default, there is no "maximum depth" for a decision tree. As such, without restriction, decision trees tend to be very deep.

In [ ]:
dt_no_max = DecisionTreeClassifier()
dt_no_max.fit(X_train, y_train)

A decision tree fit on our training data has a depth of around 20! (It is so deep that `tree.plot_tree` errors when trying to plot it.)

In [ ]:
dt_no_max.tree_.max_depth

At first, this tree seems "better" than our tree of depth 2, since its training accuracy is much much higher:

In [ ]:
dt_no_max.score(X_train, y_train)

In [ ]:
# Depth 2 tree.
dt.score(X_train, y_train)

But recall, we truly care about **test set performance**, and this decision tree has **worse accuracy on the test set than our depth 2 tree**.

In [ ]:
dt_no_max.score(X_test, y_test)

In [ ]:
# Depth 2 tree.
dt.score(X_test, y_test)

### Decision trees and overfitting

- Decision trees have a tendency to overfit. **Why is that?**

- Unlike linear classification techniques (like logistic regression or SVMs), **decision trees are non-linear**.
    - They are also "non-parametric" – there are no $w^*$s to learn.

- While being trained, decision trees ask enough questions to effectively **memorize** the correct response values in the training set. However, the relationships they learn are often overfit to the noise in the training set, and don't generalize well.

In [ ]:
fig

- A decision tree whose depth is not restricted will achieve 100% accuracy on any training set, as long as there are no "overlapping values" in the training set.
    - Two values overlap when they have the same features $x$ but different response values $y$ (e.g. if two patients have the same glucose levels and BMI, but one has diabetes and one doesn't).

- **One solution**: Make the decision tree "less complex" by limiting the maximum depth.

Since `sklearn.tree`'s `plot_tree` can't visualize extremely large decision trees, let's create and visualize some smaller decision trees.

In [ ]:
trees = {}
for d in [2, 4, 8]:
    trees[d] = DecisionTreeClassifier(max_depth=d, random_state=1)
    trees[d].fit(X_train, y_train)
    
    plt.figure(figsize=(15, 5), dpi=100)
    plot_tree(trees[d], feature_names=X_train.columns, class_names=['no db', 'yes db'], 
               filled=True, rounded=True, impurity=False)
    
    plt.show()

As tree depth increases, complexity increases, and our trees are more prone to overfitting. This means model bias decreases, but model variance increases. 

**Question**: What is the "right" maximum depth to choose?

### Hyperparameters for decision trees

- `max_depth` is a hyperparameter for `DecisionTreeClassifier`.

- There are many more hyperparameters we can tweak; look at [the documentation](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) for examples.
    - `min_samples_split`: The minimum number of samples required to split an internal node.
    - `criterion`: The function to measure the quality of a split (`'gini'` or `'entropy'`).

- To ensure that our model generalizes well to unseen data, we need an efficient technique for trying different combinations of hyperparameters!

- We'll see that technique next time: `GridSearchCV`.

## Summary, next time

### Summary

- A hyperparameter is a configuration that we choose before training a model; an important task in machine learning is selecting "good" hyperparameters.
- To choose hyperparameters, we use $k$-fold cross-validation. **This technique is very common, and you are expected to use it in Project 4!**
    - See [Summary: Generalization](#Summary:-Generalization) for more.
- Decision trees can be used for both regression and classification; we've used them for classification.
    - Decision trees are trained recursively. Each node corresponds to a "yes" or "no" question.
    - To decide which "yes" or "no" question to ask, we choose the question with the lowest weighted entropy.

### Next time

- An efficient technique for trying different combinations of hyperparameters.
    - In other words, performing $k$-fold cross-validation _without_ a `for`-loop.
- Techniques for evaluating classifiers beyond accuracy.
    - You'll need this for Project 4!